## 1. Collecting Data

### First we import the necessary libraries, then we download the data from the HM Land Registry website as:

In [2]:
import os # Operating System package
import numpy as np
import pandas as pd
import datetime as dt # Datetime module
import json # library to deal with JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # address to latitude and longitude values converter

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium for map rendering

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Reading the data for examination (Source: http://landregistry.data.gov.uk/)

In [3]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

## 2. Exploring and Understanding Data

Now read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the head as:

In [4]:
df_ppd.head(5)

,{7011B109-19A6-8ED6-E053-6B04A8C075C1},175000,2018-05-18 00:00,CW12 2DZ,S,N,F,32,Unnamed: 8,TALL ASH AVENUE,Unnamed: 10,CONGLETON,CHESHIRE EAST,CHESHIRE EAST.1,A,A.1
0,{7011B109-19A7-8ED6-E053-6B04A8C075C1},189256,2018-05-25 00:00,CW9 8QA,S,N,F,NEWLYN,NaN,JACK LANE,MOULTON,NORTHWICH,CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,A,A
1,{7011B109-19A8-8ED6-E053-6B04A8C075C1},260000,2018-05-18 00:00,WA6 0DY,D,N,F,55,NaN,LATHAM AVENUE,HELSBY,FRODSHAM,CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,A,A
2,{7011B109-19A9-8ED6-E053-6B04A8C075C1},390000,2018-05-18 00:00,CH4 8JW,D,N,F,45,NaN,HOUGH GREEN,NaN,CHESTER,CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,A,A
3,{7011B109-19AA-8ED6-E053-6B04A8C075C1},160000,2018-05-23 00:00,CW11 1HB,T,N,F,5B,NaN,VICTORIA STREET,NaN,SANDBACH,CHESHIRE EAST,CHESHIRE EAST,A,A
4,{7011B109-19AB-8ED6-E053-6B04A8C075C1},335000,2018-05-11 00:00,SK9 2NG,S,N,L,12,NaN,FIELDHEAD MEWS,NaN,WILMSLOW,CHESHIRE EAST,CHESHIRE EAST,A,A


The shape of the data is:

In [5]:
df_ppd.shape

(1032842, 16)

The dataset consists of over 1 M rows and 16 columns. We will now prepare and preprocess data accordingly.

## 3. Data Preparation and Preprocessing

Now, we can prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. For this purpose, we perform the following steps:

- Renaming the column names
- Formating the date columns
- Selecting data only for the city of London
- Sorting data by sale date
- Making a list of street names in London
- Calculating the street-wise average price of the property
- Reading the street-level coordinates into a data frame, eliminating recurring word London from individual names
- Joining the data to find the coordinates of locations which fit into client's budget
- Plotting recommended locations on London map along with current market prices

In [6]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2018
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2018].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [11]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
857,BARONSMEAD ROAD,2375000.0
...,...,...
13737,WILFRED STREET,2410538.5
13763,WILLOW BRIDGE ROAD,2425000.0
13783,WILSON STREET,2257500.0
13812,WINCHENDON ROAD,2350000.0


In [12]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
#from geopy.distance import vincenty
from geopy.distance import geodesic
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [17]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 857
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 857, dtype: object
item.Street only: BARONSMEAD ROAD
index: 983
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 983, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1105
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1105, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1218
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [18]:
geolocator = Nominatim(user_agent="my_application")

In [19]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

C:\AnaConda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)"
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)"
422,APPLEGARTH ROAD,2400000.0,"(40.2746238, -74.4723052)"
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"
...,...,...,...
13737,WILFRED STREET,2410538.5,"(51.442178, 0.372743)"
13763,WILLOW BRIDGE ROAD,2425000.0,"(39.6415648, -106.3757744)"
13783,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)"
13812,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)"


In [21]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

C:\AnaConda\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [22]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)",29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)",51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,"(40.2746238, -74.4723052)",40.274624,-74.472305
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457
...,...,...,...,...,...
13737,WILFRED STREET,2410538.5,"(51.442178, 0.372743)",51.442178,0.372743
13763,WILLOW BRIDGE ROAD,2425000.0,"(39.6415648, -106.3757744)",39.641565,-106.375774
13783,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)",45.700486,-121.521875
13812,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)",42.645331,-71.954714


In [23]:
df = df_affordable.drop(columns=['city_coord'])

In [24]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.274624,-74.472305
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457
...,...,...,...,...
13737,WILFRED STREET,2410538.5,51.442178,0.372743
13763,WILLOW BRIDGE ROAD,2425000.0,39.641565,-106.375774
13783,WILSON STREET,2257500.0,45.700486,-121.521875
13812,WINCHENDON ROAD,2350000.0,42.645331,-71.954714


In [25]:
address = 'London, UK'

#geolocator = Nominatim()
geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [26]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [1]:
#Define Foursquare Credentials and Version

CLIENT_ID = '(*********************************)' # Foursquare ID
CLIENT_SECRET = '******************************' # Foursquare Secret
VERSION = '20210621' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: (*********************************)
CLIENT_SECRET:******************************


We now proceed to the modeling phase in which we analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

## 4. Modeling

Since we have explored the dataset and gained insights into it, we are ready to use the clustering methodology to analyze real estates. We can use the k-means clustering technique as it is fast and efficient in terms of computational cost, accurate, highly flexible to account for mutations in real estate market in London.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE

In [30]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
3,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
4,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
...,...,...,...,...,...,...,...
4574,WILSON STREET,45.700486,-121.521875,The Mesquitery,45.700912,-121.523445,BBQ Joint
4575,WILSON STREET,45.700486,-121.521875,ECHO River Trips,45.701159,-121.520677,River
4576,WILSON STREET,45.700486,-121.521875,Juanita's Marketa,45.700125,-121.524078,Grocery Store
4577,WILSON STREET,45.700486,-121.521875,Solstice Pop Up Pizza & Mobile Kitchen,45.699242,-121.523825,Food Truck


In [31]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,18,18,18,18,18,18
ANSDELL TERRACE,57,57,57,57,57,57
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,2,2,2,2,2,2
...,...,...,...,...,...,...
WESTMORELAND PLACE,17,17,17,17,17,17
WHITFIELD STREET,9,9,9,9,9,9
WILFRED STREET,26,26,26,26,26,26


In [32]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 333 uniques categories.


In [33]:
location_venues.shape

(4579, 7)

In [34]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ALBION SQUARE,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,ANSDELL TERRACE,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.017544,0.0,0.0,0.0,0.0
2,APPLEGARTH ROAD,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,BARONSMEAD ROAD,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,BEAUCLERC ROAD,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,WESTMORELAND PLACE,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.117647,0.0,...,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
144,WHITFIELD STREET,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
145,WILFRED STREET,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.038462,0.038462,0.000000,0.000000,0.0,0.0,0.0,0.0
146,WILLOW BRIDGE ROAD,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.122449,0.0,...,0.000000,0.0,0.000000,0.000000,0.020408,0.020408,0.0,0.0,0.0,0.0


In [36]:
london_grouped.shape

(148, 334)

In [37]:
# What are the top 5 venues/facilities nearby profitable real estate investments?

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
         venue  freq
0         Café  0.22
1   Restaurant  0.11
2          Bar  0.11
3          Pub  0.11
4  Art Gallery  0.06


----ANSDELL TERRACE----
                venue  freq
0          Restaurant  0.07
1  Italian Restaurant  0.07
2   Indian Restaurant  0.07
3           Juice Bar  0.05
4               Hotel  0.05


----APPLEGARTH ROAD----
                  venue  freq
0     Indian Restaurant  0.25
1             Pet Store  0.25
2                  Food  0.25
3      Asian Restaurant  0.25
4  Pakistani Restaurant  0.00


----BARONSMEAD ROAD----
               venue  freq
0         Restaurant  0.08
1    Thai Restaurant  0.08
2                Pub  0.08
3  Convenience Store  0.08
4        Coffee Shop  0.08


----BEAUCLERC ROAD----
                     venue  freq
0  Health & Beauty Service   0.5
1                Locksmith   0.5
2                      ATM   0.0
3     Pakistani Restaurant   0.0
4         Pedestrian Plaza   0.0


----BELVEDERE DRIVE----
               

In [38]:
# Let's define a function to return the most common venues/facilities nearby real estate investments

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [40]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [41]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Pub,Restaurant,Bar,Fish & Chips Shop,Supermarket,Beer Garden,Park,River,Art Gallery
1,ANSDELL TERRACE,Indian Restaurant,Restaurant,Italian Restaurant,Hotel,Juice Bar,Grocery Store,Japanese Restaurant,Bakery,Pub,Coffee Shop
2,APPLEGARTH ROAD,Food,Pet Store,Indian Restaurant,Asian Restaurant,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
3,BARONSMEAD ROAD,Breakfast Spot,Restaurant,Pub,Sports Club,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
4,BEAUCLERC ROAD,Health & Beauty Service,Locksmith,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot


In [42]:
venues_sorted.shape

(148, 11)

In [43]:
london_grouped.shape

(148, 334)

In [44]:
london_grouped=df

Now we have inspected the venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [45]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 2, 4, 0, 1, 0, 2, 2, 1, 4, 4, 1, 0, 0, 1, 4, 2, 1, 3, 3, 4,
       1, 1, 2, 4, 3, 1, 2, 4, 0, 4, 0, 0, 3, 4, 4, 0, 2, 1, 0, 3, 2, 3,
       2, 2, 3, 2, 2, 4])

In [46]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.274624,-74.472305
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [47]:
london_grouped_clustering.shape

(159, 4)

In [48]:
df.shape

(159, 4)

In [49]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [50]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [51]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Pub,Restaurant,Bar,Fish & Chips Shop,Supermarket,Beer Garden,Park,River,Art Gallery
390,ANHALT ROAD,2.435000e+06,29.805547,-98.475534,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,ANSDELL TERRACE,2.250000e+06,51.500005,-0.189154,2,Indian Restaurant,Restaurant,Italian Restaurant,Hotel,Juice Bar,Grocery Store,Japanese Restaurant,Bakery,Pub,Coffee Shop
422,APPLEGARTH ROAD,2.400000e+06,40.274624,-74.472305,4,Food,Pet Store,Indian Restaurant,Asian Restaurant,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
857,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,0,Breakfast Spot,Restaurant,Pub,Sports Club,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
983,BEAUCLERC ROAD,2.480000e+06,30.207887,-81.628209,1,Health & Beauty Service,Locksmith,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot
1105,BELVEDERE DRIVE,2.340000e+06,38.242587,-77.382574,0,Theme Park,Racetrack,Farm,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
1218,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,2,Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Café,Chinese Restaurant,Bar,Greek Restaurant,Gastropub,Movie Theater
1256,BIRCHLANDS AVENUE,2.217000e+06,51.448414,-0.160457,2,Pub,Breakfast Spot,Lake,French Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Train Station,Brewery,Pizza Place
1556,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394,1,Middle Eastern Restaurant,Lake,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot


In [52]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
857,2375000.0,Breakfast Spot,Restaurant,Pub,Sports Club,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
1105,2340000.0,Theme Park,Racetrack,Farm,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
2071,2375000.0,Pub,Grocery Store,Italian Restaurant,Indian Restaurant,Pizza Place,Park,Hotel,Hostel,Greek Restaurant,Bubble Tea Shop
2132,2379652.7,Pub,Grocery Store,Pizza Place,Hotel,Italian Restaurant,Hostel,Indian Restaurant,Coffee Shop,Sandwich Place,Record Shop
2947,2367500.0,Hotel,Pub,Café,Indian Restaurant,Coffee Shop,Italian Restaurant,Cocktail Bar,Chinese Restaurant,French Restaurant,Bakery


In [54]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Pub,Restaurant,Bar,Fish & Chips Shop,Supermarket,Beer Garden,Park,River,Art Gallery
983,2480000.0,Health & Beauty Service,Locksmith,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot
1556,2456875.0,Middle Eastern Restaurant,Lake,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot
1983,2492500.0,Supermarket,English Restaurant,Dry Cleaner,Discount Store,Gym,Hardware Store,Fast Food Restaurant,Coffee Shop,Café,Park
2139,2461000.0,Pub,Fish & Chips Shop,Yoga Studio,Food,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Fishing Spot


In [55]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Indian Restaurant,Restaurant,Italian Restaurant,Hotel,Juice Bar,Grocery Store,Japanese Restaurant,Bakery,Pub,Coffee Shop
1218,2208500.0,Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Café,Chinese Restaurant,Bar,Greek Restaurant,Gastropub,Movie Theater
1256,2217000.0,Pub,Breakfast Spot,Lake,French Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Train Station,Brewery,Pizza Place
2228,2200000.0,Construction & Landscaping,Hotel,Yoga Studio,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
2641,2250000.0,Café,Pub,Garden,English Restaurant,Grocery Store,Coffee Shop,Pizza Place,Tapas Restaurant,Gym / Fitness Center,Supermarket


In [56]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2245,2.300000e+06,Farm,Yoga Studio,Factory,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flower Shop
2409,2.286679e+06,Pub,Café,Coffee Shop,Italian Restaurant,Bar,Park,Convenience Store,Grocery Store,Yoga Studio,Hotel
2689,2.287500e+06,Pub,Brewery,Hotel,Music Venue,Art Museum,Reservoir,Friterie,Food,Farmers Market,Fast Food Restaurant
3380,2.298000e+06,Hotel,Yoga Studio,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot
4288,2.265000e+06,Fast Food Restaurant,Gym / Fitness Center,Pub,Convenience Store,Grocery Store,Pizza Place,Yoga Studio,Fishing Spot,Farm,Farmers Market


In [57]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,2400000.0,Food,Pet Store,Indian Restaurant,Asian Restaurant,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
1635,2397132.0,Coffee Shop,Italian Restaurant,Sporting Goods Shop,Park,Convenience Store,Grocery Store,Yoga Studio,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
1800,2400000.0,Art Gallery,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flower Shop
2161,2425000.0,Hotel,Italian Restaurant,Clothing Store,Pub,Coffee Shop,French Restaurant,Chinese Restaurant,Bar,Restaurant,Pizza Place


## Results and Discussion:

Even though the London Housing Market is facing a temporary bad times, it is still an "ever-green" for business affairs. We may elaborate our results under two main perspectives:

First Perspective: 

One can examine them according to neighborhoods. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. In the same way, one might target under-priced real estates in these areas of London in order to make a business affair.

Second Perspective:

One can analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities. We found two main patterns, i.e. the first pattern is Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern is Clusters 1 and 3, may target individuals who love pubs, theatres and soccer type activities.

## Conclusion Remarks:

Summerizing the project we conclude that according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

The solution to this business problem was that we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

Firstly, we collected data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Secondly, The Data Methodology section consisted upon four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we concluded that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods. Although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.